# Prediction Algorithms

## Preliminaries

In [67]:
import os
import sys
import networkx as nx
import numpy as np

from os.path import dirname
from networkx.drawing.nx_pydot import to_pydot

note_dir = os.getcwd()

root_dir = dirname(note_dir)
src_dir = os.path.join(root_dir, "src")

sys.path.append(src_dir)

import morpheus
from morpheus.utils.encoding import code_to_query
from morpheus.algo.selection import base_selection_algorithm
from morpheus.tests import default_chain, default_ensemble, default_dataset, learn_model

## Collect Basic Building Blocks

In [42]:
X, _ = default_dataset(random_state=997)

(800, 8)

In [72]:
data = X.values

n, m = data.shape
n, m

(800, 8)

In [46]:
metadata = {'nb_atts': m}
settings = {'param':   1,
            'its':     1}

m_codes = base_selection_algorithm(metadata, settings)
m_codes

array([[0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.]])

In [48]:
m_code = m_codes[0, :]
m_code

array([0., 1., 0., 0., 0., 0., 0., 0.])

In [56]:
code_to_query(m_code)


([0, 2, 3, 4, 5, 6, 7], [1], [])

In [64]:
all_desc_ids, all_targ_ids = [], []
for m_code in m_codes:
    desc_ids, targ_ids, _ = code_to_query(m_code)
    all_desc_ids.append(desc_ids)
    all_targ_ids.append(targ_ids)

m_list = []
ids = zip(all_desc_ids, all_targ_ids)
ids

In [71]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
for desc_ids, targ_ids in ids:
    msg = """
    Learning model with desc ids:    {}
                        targ ids:    {}
    """.format(
        desc_ids, targ_ids
    )
    print(msg)

    if set(targ_ids).issubset({6, 7}):
        learner = RandomForestClassifier
    elif set(targ_ids).issubset({0, 1, 2, 3, 4, 5}):
        learner = RandomForestRegressor
    else:
        msg = """
        Cannot learn mixed (nominal/numeric) models
        """
        raise ValueError(msg)

    # Learn a model for desc_ids-targ_ids
    m = learn_model(data, desc_ids, targ_ids, learner, max_depth=5, n_estimators=5)
    m_list.append(m)


    Learning model with desc ids:    [1, 2, 3, 4, 5, 6, 7]
                        targ ids:    [0]
    

    Learning model with desc ids:    [0, 1, 2, 3, 4, 5, 6]
                        targ ids:    [7]
    

    Learning model with desc ids:    [0, 1, 3, 4, 5, 6, 7]
                        targ ids:    [2]
    

    Learning model with desc ids:    [0, 1, 2, 4, 5, 6, 7]
                        targ ids:    [3]
    

    Learning model with desc ids:    [0, 1, 2, 3, 5, 6, 7]
                        targ ids:    [4]
    


## MI

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])